# Sequence Modeling: Recurrent and Recursive Nets

**Recurrent neural networks**, or RNNs, are a family of neural networks for processing sequential data. Much as a convolutional network is a neural network that is specialized for processing a grid of values $X$ such as an image, a recurrent neural network is a neural network that is specialized for processing a sequence of values $x^{(1)},... , x^{(n)}$. Just as convolutional networks can readily scale to images with large width and height, and some convolutional networks can process images of variable size, recurrent networks can scale to much longer sequences than would be practical for networks without sequence-based specialization. Most recurrent networks can also process sequences of variable length.

To go from multilayer networks to recurrent networks, we need to take advantage of one of the early ideas found in machine learning and statistical models of the 1980s: sharing parameters across different parts of a model. Parameter sharingm akes it possible to extend and apply the model to examples of different forms(diﬀerent lengths, here) and generalize across them.

For example, consider the two sentences “I went to Nepal in 2009” and “In 2009,I went to Nepal.” If we ask a machine learning model to read each sentence andextract the year in which the narrator went to Nepal, we would like it to recognizethe year 2009 as the relevant piece of information, whether it appears in the sixth word or in the second word of the sentence. Suppose that we trained a feedforward network that processes sentences of ﬁxed length. A traditional fully connected feedforward network would have separate parameters for each input feature, so it would need to learn all the rules of the language separately at each position in the sentence. By comparison, a recurrent neural network shares the same weights across several time steps.

For the simplicity of exposition, we refer to RNNs as operating on a sequence that contains vectors $x^{(t)}$ with the time step indext ranging from 1 to $\tau$. Each member of the output is a function of the previous members of the output. Each member of the output is produced using the same update rule applied to the previous outputs.This recurrent formulation results in the sharing of parameters through a very deep computational graph.

## Unfolding Computational Graphs

A computational graph is a way to formalize the structure of a set of computations,such as those involved in mapping inputs and parameters to outputs and loss. **Unfolding** this graph results in the sharing of parameters across a deep network structure. For example, consider the classical form of a dynamical system:

<img src="img/ex1.png">

$$
h^{(t)} = f(h^{(t-1)}; \theta)
$$

Where $s^{(t)}$ is called the state of the system. The classical dynamical system described by equation above, illustrated as an unfolded computational graph. Each node represents the state at some time $t$, and the function $f$ maps the state at $t$ to the state at $t+ 1$. The same parameters (the same valueof $\theta$ used to parametrize $f$) are used for all time steps.

As another example, let us consider a dynamical system driven by an external signal $x^{(t)}$,

$$
h^{(t)} = f(h^{(t-1)}, x^{(t)}; \theta)
$$

where we see that the state now contains information about the whole past sequence.

<img src="img/ex2.png">

A recurrent network with no outputs. This recurrent network just processes information from the input $x$ by incorporating it into the state $h$ that is passed forward through time. (Left) Circuit diagram. The black square indicates a delay of a single timestep. (Right) The same network seen as an unfolded computational graph, where each node is now associated with one particular time instance.

Recurrent neural networks can be built in many diﬀerent ways. Much as almost any function can be considered a feedforward neural network, essentially any function involving recurrence can be considered a recurrent neural network.

We can represent the unfolded recurrence after t steps with a function $g^{(t)}$:

$$
h^{(t)} = g^{(t)}(x^{(t)}, x^{(t-1)},...,x^{(1)}) = f(h^{(t-1)}, x^{(t)}; \theta)
$$

The function $g^{(t)}$ takes the whole past sequence $(x^{(t)}, x^{(t-1)},...,x^{(1)})$ as input and produces the current state, but the unfolded recurrent structure allows us to factorize  $g^{(t)}$ into repeated application of a function $f$. The unfolding process thus introduces two major advantages:
1. Regardless of the sequence length, the learned model always has the sameinput size, because it is speciﬁed in terms of transition from one state toanother state, rather than speciﬁed in terms of a variable-length history ofstates.
2. It is possible to use the same transition functionfwith the same parametersat every time step

These two factors make it possible to learn a single model $f$ that operates on all time steps and all sequence lengths, rather than needing to learn a separate model $g^{(t)}$ for all possible time steps.

## Recurrent Neural Networks

Below is a figure of a easonably representative example that we return to throughout most of the notebook

<img src="img/ex3.png">

The computational graph to compute the training loss of a recurrent network that maps an input sequence of $x$ values to a corresponding sequence of output $o$ values.A loss $L$ measures how far each $o$ is from the corresponding training target $y$. When using softmax outputs, we assume $o$ is the unnormalized log probabilities. The loss $L$ internally computes $\hat{y}=softmax(o)$ and compares this to the target $y$. The RNN has input to hidden connections parametrized by a weight matrix $U$, hidden-to-hidden recurrent connections parametrized by a weight matrix $W$, and hidden-to-output connections parametrized by a weight matrix $V$. (Left)The RNN and its loss drawn with recurrent connections. (Right) The same seen as a time-unfolded computational graph, where each node is now associated with one particular time instance.

We now develop the forward propagation equations for the RNN depicted in ﬁgure above. The ﬁgure does not specify the choice of activation function for the hidden units. Here we assume the hyperbolic tangent activation function. Also,the ﬁgure does not specify exactly what form the output and loss function take.Here we assume that the output is discrete, as if the RNN is used to predict wordsor characters. we apply the following update equations

$$
a^{(t)} = b+Wh^{(t-1)} + Ux^{(t)}
$$
$$
h^{(t)} = tanh(a^{(t)})
$$
$$
o^{(t)} = c + Vh^{(t)}
$$
$$
\hat{y}^{(t)} = softmax(o^{(t)})
$$


where the parameters are the bias vectors b and c along with the weight matrices U, V and W, respectivly, for input-to-hidden, hidden-to-output and hidden-to-hidden connections. This is an example of a recurrent network that maps an input sequence to an output sequence of the same length. The total loss for a given sequence of x values paired with a sequence of y values would then be just the sum of the losses over all the time steps. 

**Example**: If we let $L^{(t)}$ is the negative log-likelihood of $y^{(t)}$ given $x^{(1)},...,x^{(t)}$, then
$$
L\left({x^{(1)},...,x^{(\tau)}, y^{(1)},...,y^{(\tau)}}\right) = \sum_t L^{(t)}
 = -\sum log p_{model}\left(y^{(t)}| x^{(1)},...,x^{(t)} \right)
$$

<img src="img/ex4.png">

An RNN whose only recurrence is the feedback connection from the output to the hidden layer. At each time stept, the input is $x_t$, the hidden layer activationsare $h^{(t)}$, the outputs are $o^{(t)}$, the targets are $y^{(t)}$, and the loss is $L^{(t)}$. (Left) Circuit diagram. (Right) Unfolded computational graph. The RNN in this ﬁgure is trained to put a speciﬁc output value in to $o$, and $o$ is the only information it is allowed to send to the future. There are no direct connections from $h$ going forward. The previous $h$ is connected to the present only indirectly, via the predictions it was used to produce. Unless $o$ is very high-dimensional and rich, it will usually lack important information from the past. This makes the RNN in this ﬁgure less powerful, but it may be easier to train because each time step can be trained in isolation from the others, allowing greater parallelization during training.

<img src="img/ex5.png">

Time-unfolded recurrent neural network with a single output at the end of the sequence. Such a network can be used to summarize a sequence and produce aﬁxed-size representation used as input for further processing. There might be a target right at the end (as depicted here), or the gradient on the output $o^{(t)}$ can be obtained by back-propagating from further downstream modules.